In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import shutil
import os
import random
import cv2
import numpy as np
import re
import copy
from typing import List, Dict, Any, Optional, Tuple
import math
import matplotlib.pyplot as plt

import numpy as np

import PIL
from PIL import Image, ImageChops, ImageEnhance, ImageFilter, ImageOps

In [2]:
!unzip /content/rec.zip

Archive:  /content/rec.zip
   creating: rec/
   creating: rec/configs/
   creating: rec/configs/dataset/
   creating: rec/configs/dataset/rec/
  inflating: rec/configs/dataset/rec/evaluation.yaml  
  inflating: rec/configs/dataset/rec/ltb.yaml  
  inflating: rec/configs/dataset/rec/mjsynth.yaml  
  inflating: rec/configs/dataset/rec/openvino.yaml  
  inflating: rec/configs/dataset/rec/ost.yaml  
  inflating: rec/configs/dataset/rec/synthtext.yaml  
  inflating: rec/configs/dataset/rec/test.yaml  
  inflating: rec/configs/dataset/rec/textocr.yaml  
  inflating: rec/configs/dataset/rec/textocr_horizontal.yaml  
  inflating: rec/configs/dataset/rec/union14m_b.yaml  
  inflating: rec/configs/dataset/rec/union14m_image.yaml  
  inflating: rec/configs/dataset/rec/union14m_l_filtered.yaml  
   creating: rec/configs/rec/
  inflating: rec/configs/rec/focalsvtr_smtr.yml  
   creating: rec/data/
   creating: rec/data/.ipynb_checkpoints/
  inflating: rec/data/aug_image.py   
  inflating: rec/data/

In [3]:
!unzip /content/OpenOCR.zip

Archive:  /content/OpenOCR.zip
   creating: OpenOCR/
  inflating: OpenOCR/.gitignore      
  inflating: OpenOCR/.pre-commit-config.yaml  
   creating: OpenOCR/configs/
   creating: OpenOCR/configs/dataset/
   creating: OpenOCR/configs/dataset/rec/
  inflating: OpenOCR/configs/dataset/rec/evaluation.yaml  
  inflating: OpenOCR/configs/dataset/rec/ltb.yaml  
  inflating: OpenOCR/configs/dataset/rec/mjsynth.yaml  
  inflating: OpenOCR/configs/dataset/rec/openvino.yaml  
  inflating: OpenOCR/configs/dataset/rec/ost.yaml  
  inflating: OpenOCR/configs/dataset/rec/synthtext.yaml  
  inflating: OpenOCR/configs/dataset/rec/test.yaml  
  inflating: OpenOCR/configs/dataset/rec/textocr.yaml  
  inflating: OpenOCR/configs/dataset/rec/textocr_horizontal.yaml  
  inflating: OpenOCR/configs/dataset/rec/union14m_b.yaml  
  inflating: OpenOCR/configs/dataset/rec/union14m_l_filtered.yaml  
   creating: OpenOCR/configs/det/
   creating: OpenOCR/configs/det/dbnet/
  inflating: OpenOCR/configs/det/dbnet/re

In [4]:
!pip install -r /content/OpenOCR/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 90.5 MB/s eta 0:00:00


In [5]:
!pip install python-magic

# Download VietNamese data

In [6]:
%cd rec

/content/rec


In [7]:
!python /content/rec/data/download/download_dataset.py -c /content/OpenOCR/configs/dataset/rec/ltb.yaml


[1/2] Downloading ../ltb/data.mdb from https://drive.usercontent.google.com/download?id=16AEA1YGTsyVB44uEjKi4ZUV1snjCYBr4&authuser=0&confirm=t ...
100% 74.4M/74.4M [00:00<00:00, 240MB/s]
Successfully downloaded ../ltb/data.mdb

[2/2] Downloading ../ltb/lock.mdb from https://drive.usercontent.google.com/download?id=1xU4OStrOaI23bPG4flWAPWn2YrQe2bmY&authuser=0&confirm=t ...
100% 8.19k/8.19k [00:00<00:00, 28.3MB/s]
Successfully downloaded ../ltb/lock.mdb
Downloads and extractions finished!


In [8]:
!python /content/rec/data/download/download_dataset.py -c /content/rec/configs/dataset/rec/union14m_l_filtered.yaml


[1/13] Downloading ../Union14M-L-LMDB-Filtered/train_challenging/data.mdb from https://drive.usercontent.google.com/download?id=1etwzBgGHjsFsb0sygsaRnKbanW2PMe07&authuser=0&confirm=t ...
100% 1.56G/1.56G [00:17<00:00, 88.9MB/s]
Successfully downloaded ../Union14M-L-LMDB-Filtered/train_challenging/data.mdb

[2/13] Downloading ../Union14M-L-LMDB-Filtered/train_challenging/lock.mdb from https://drive.usercontent.google.com/download?id=1ly6FJfPjItwGlVQ-ifTrzzM3rVu3Ezhr&authuser=0&confirm=t ...
100% 8.19k/8.19k [00:00<00:00, 27.8MB/s]
Successfully downloaded ../Union14M-L-LMDB-Filtered/train_challenging/lock.mdb

[3/13] Downloading ../Union14M-L-LMDB-Filtered/train_easy/data.mdb from https://drive.usercontent.google.com/download?id=1_zeNluTnywIaa5h3PN-Ah9tKyByypot7&authuser=0&confirm=t ...
100% 6.43G/6.43G [00:53<00:00, 121MB/s]
Successfully downloaded ../Union14M-L-LMDB-Filtered/train_easy/data.mdb

[4/13] Downloading ../Union14M-L-LMDB-Filtered/train_easy/lock.mdb from https://drive.user

In [9]:
!python /content/rec/data/download/download_dataset.py -c /content/rec/configs/dataset/rec/evaluation.yaml


[1/12] Downloading ../evaluation/IC15_1811/data.mdb from https://drive.usercontent.google.com/download?id=1eGY0kXNV1qVxeUpoGzs-ioUO-ky7msH6&authuser=0&confirm=t ...
100% 21.9M/21.9M [00:00<00:00, 137MB/s] 
Successfully downloaded ../evaluation/IC15_1811/data.mdb

[2/12] Downloading ../evaluation/IC15_1811/lock.mdb from https://drive.usercontent.google.com/download?id=1BWv7aLoLAT7avY326gXP3GJF48UZpuBC&authuser=0&confirm=t ...
100% 8.19k/8.19k [00:00<00:00, 26.9MB/s]
Successfully downloaded ../evaluation/IC15_1811/lock.mdb

[3/12] Downloading ../evaluation/SVT/data.mdb from https://drive.usercontent.google.com/download?id=1ecEZ4cJ7dIbTCZRltE0s5KzUotQWagH-&authuser=0&confirm=t ...
100% 6.79M/6.79M [00:00<00:00, 114MB/s]
Successfully downloaded ../evaluation/SVT/data.mdb

[4/12] Downloading ../evaluation/SVT/lock.mdb from https://drive.usercontent.google.com/download?id=1OygBP7i9R-3Pwi6WodCcW31J8CUMugOJ&authuser=0&confirm=t ...
100% 8.19k/8.19k [00:00<00:00, 26.0MB/s]
Successfully downloa

In [10]:
# @title Create lmdb VNese dataset
import json

def read_crop_label(filepaths, output_filename):
    """
    Đọc dữ liệu từ các file, tạo file JSON với định dạng:
    {"filename": "path/to/image", "text": "label"}
    """
    n_lines = 0
    samples = 0
    try:
        with open(output_filename, 'w', encoding='utf-8') as f_out:
            for filepath in filepaths:
                try:
                    with open(filepath, 'r', encoding='utf-8') as f_in:
                        for line in f_in:
                            n_lines += 1
                            line = line.strip()
                            if line:
                                parts = line.split('\t')
                                if len(parts) == 2:
                                    image_path, label = parts
                                    if len(label) > 200 or len(label) == 0:
                                        # print(f"Dòng len(label) > 200: {line}, len = {len(label)}")
                                        continue
                                    json_data = {"filename": image_path, "text": label}
                                    json.dump(json_data, f_out, ensure_ascii=False)
                                    f_out.write('\n')
                                    samples += 1
                                # else:
                                #     print(f"Dòng không hợp lệ: {line}, len = {len(parts)}")
                except FileNotFoundError:
                    print(f"Không tìm thấy file: {filepath}")
                except Exception as e:
                    print(f"Lỗi khi đọc file {filepath}: {e}")
        print(f"Đã tạo file JSON: {output_filename}")
    except Exception as e:
        print(f"Lỗi tổng quan: {e}")
    return n_lines, samples

# Sử dụng hàm
filepath = ["/content/Raw_vnese_data/data/crop_label.txt",
            "/content/Raw_vnese_data/vietocr-dataset/VietOCR-Paddle/InkData_line_processed/rec/rec_gt_train_ink.txt",
            "/content/Raw_vnese_data/vietocr-dataset/VietOCR-Paddle/en_00/rec/rec_gt_train_en_00.txt",
            "/content/Raw_vnese_data/vietocr-dataset/VietOCR-Paddle/en_01/rec/rec_gt_train_en01.txt",
            "/content/Raw_vnese_data/vietocr-dataset/VietOCR-Paddle/meta/rec/rec_gt_train_meta.txt",
            "/content/Raw_vnese_data/vietocr-dataset/VietOCR-Paddle/random/rec/rec_gt_train_random.txt",
            "/content/Raw_vnese_data/vietocr-dataset/VietOCR-Paddle/vi_00/rec/rec_gt_train_vi00.txt",
            "/content/Raw_vnese_data/vietocr-dataset/VietOCR-Paddle/vi_01/rec/rec_gt_train_vi01.txt"]

output_filename = "/content/vnese.json"
n_lines, samples = read_crop_label(filepath, output_filename)
print(f"Tổng số dòng: {n_lines}")     # Tổng số dòng: 702152
print(f"Tổng số mẫu hợp lệ samples: {samples}")     # Tổng số mẫu hợp lệ samples: 701870

Đã tạo file JSON: /content/vnese.json
Tổng số dòng: 702152
Tổng số mẫu hợp lệ samples: 701870


In [11]:
!python /content/rec/data/create_lmdb_dataset.py \
-l /content/vnese.json \
-s /content/Union14M-L-LMDB-Filtered/train_vnese \
-m 200

/content/Union14M-L-LMDB-Filtered/train_vnese
load data from /content/vnese.json: 100% 701870/701870 [00:02<00:00, 308762.82it/s]
make dataset, save to /content/Union14M-L-LMDB-Filtered/train_vnese: 100% 701870/701870 [04:07<00:00, 2830.66it/s]
Created dataset with 701870 samples


# Load github repository

In [22]:
%cd /content/OpenOCR

/content/OpenOCR


# Training

In [13]:
!pip install "numpy<2.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 103.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


In [14]:
!python -m torch.distributed.launch\
 --nproc_per_node=1 tools/train_rec.py\
 --c /content/OpenOCR/configs/rec/svtrv2/svtrv2_smtr_gtc_rctc_small.yml\
#  --o Global.pretrained_model=./output/rec/u14m_filter_vnese/svtrv2_smtr_gtc_rctc_small/best.pth


Streaming output truncated to the last 5000 lines.
[2025/04/02 13:32:32] openrec INFO: epoch: [6/8], global_step: 84100, lr: 0.000132, acc: 0.817187, norm_edit_dis: 0.930884, num_samples: 320.000000, gtc_acc: 0.795312, gtc_norm_edit_dis: 0.920598, loss: 0.510611, ctc_loss: 2.091443, gtc_loss: 0.306780, avg_reader_cost: 0.01981 s, avg_batch_cost: 0.33850 s, avg_samples: 234.6, ips: 693.06435 samples/s, eta: 4:30:20
[2025/04/02 13:32:35] openrec INFO: epoch: [6/8], global_step: 84110, lr: 0.000132, acc: 0.757912, norm_edit_dis: 0.913318, num_samples: 320.000000, gtc_acc: 0.757629, gtc_norm_edit_dis: 0.861450, loss: 0.757950, ctc_loss: 2.399703, gtc_loss: 0.353267, avg_reader_cost: 0.02206 s, avg_batch_cost: 0.33066 s, avg_samples: 246.6, ips: 745.77566 samples/s, eta: 4:30:16
[2025/04/02 13:32:39] openrec INFO: epoch: [6/8], global_step: 84120, lr: 0.000132, acc: 0.790449, norm_edit_dis: 0.928463, num_samples: 320.000000, gtc_acc: 0.768750, gtc_norm_edit_dis: 0.890384, loss: 0.757950, ct

In [26]:
!python tools/eval_rec.py --c configs/rec/svtrv2/svtrv2_smtr_gtc_rctc_small.yml

[2025/04/02 18:27:43] openrec INFO: ----------- Config -----------
[2025/04/02 18:27:43] openrec INFO: Architecture : 
[2025/04/02 18:27:43] openrec INFO:     Decoder : 
[2025/04/02 18:27:43] openrec INFO:         ctc_decoder : 
[2025/04/02 18:27:43] openrec INFO:             name : RCTCDecoder
[2025/04/02 18:27:43] openrec INFO:         detach : False
[2025/04/02 18:27:43] openrec INFO:         gtc_decoder : 
[2025/04/02 18:27:43] openrec INFO:             ds : True
[2025/04/02 18:27:43] openrec INFO:             max_len : 200
[2025/04/02 18:27:43] openrec INFO:             name : SMTRDecoder
[2025/04/02 18:27:43] openrec INFO:             next_mode : True
[2025/04/02 18:27:43] openrec INFO:             num_layer : 1
[2025/04/02 18:27:43] openrec INFO:             sub_str_len : 5
[2025/04/02 18:27:43] openrec INFO:         infer_gtc : True
[2025/04/02 18:27:43] openrec INFO:         name : GTCDecoder
[2025/04/02 18:27:43] openrec INFO:     Encoder : 
[2025/04/02 18:27:43] openrec INFO

In [19]:
%cd /content

/content


In [20]:
!zip -r my_project.zip /content/OpenOCR

updating: content/OpenOCR/ (stored 0%)
updating: content/OpenOCR/__init__.py (deflated 47%)
updating: content/OpenOCR/opendet/ (stored 0%)
updating: content/OpenOCR/opendet/postprocess/ (stored 0%)
updating: content/OpenOCR/opendet/postprocess/__init__.py (deflated 51%)
updating: content/OpenOCR/opendet/postprocess/db_postprocess.py (deflated 75%)
updating: content/OpenOCR/opendet/metrics/ (stored 0%)
updating: content/OpenOCR/opendet/metrics/__init__.py (deflated 47%)
updating: content/OpenOCR/opendet/metrics/det_metric.py (deflated 76%)
updating: content/OpenOCR/opendet/metrics/eval_det_iou.py (deflated 75%)
updating: content/OpenOCR/opendet/modeling/ (stored 0%)
updating: content/OpenOCR/opendet/modeling/__init__.py (deflated 38%)
updating: content/OpenOCR/opendet/modeling/backbones/ (stored 0%)
updating: content/OpenOCR/opendet/modeling/backbones/__init__.py (deflated 44%)
updating: content/OpenOCR/opendet/modeling/backbones/repvit.py (deflated 75%)
updating: content/OpenOCR/opende